# Project #1 Data Visualization

## Import Packages

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import re

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

In [ ]:
import plotly.io as pio

## Where's the Data From?

Data is from the following Kaggle Link

https://www.kaggle.com/datasets/vincentcornlius/sales-orders?resource=download

## Import Data

In [ ]:
#Make sure to update with your path
Path = r"/Users/peterlyon/Documents/Jupyter Notebooks/sales_data.csv"
df1 = pd.read_csv(os.path.normpath(Path))
df1

## Data Cleaning

In [ ]:
df1["State Abrev"] = df1["Purchase Address"].str[-8:-6]
df1["ZIP"] = df1["Purchase Address"].str[-5:]

In [ ]:
df1['Order Date'] = pd.to_datetime(df1['Order Date'])

In [ ]:
df1['Day'] = df1['Order Date'].dt.day
df1['Month'] = df1['Order Date'].dt.month
df1['Year'] = df1['Order Date'].dt.year

In [ ]:
df1.rename(columns={'catégorie': 'Category'}, inplace = True)
df1.rename(columns={'Order Date': 'Order Date and Time'}, inplace = True)
df1.rename(columns={'turnover': 'Turnover'}, inplace = True)
df1.rename(columns={'margin': 'Margin'}, inplace = True)

In [ ]:
df1['Total Sales'] = df1['Quantity Ordered'] * df1['Price Each']
df1

In [ ]:
df2 = df1["Purchase Address"].str.rsplit(",", n=2, expand=True)
df2

In [ ]:
df1["City"] = df2[1]
df1["Local Address"] = df2[0]

In [ ]:
df1['Week Number'] = df1['Order Date and Time'].dt.isocalendar().week
df1

In [ ]:
Daily_Sales_All = df1.groupby(['Year', 'Month', 'Day', 'Week Number']).agg({'Price Each': 'sum', 'Quantity Ordered': 'sum', 'Turnover': 'sum', 'Margin': 'sum', 'Total Sales': 'sum'}).reset_index()
Daily_Sales_All

In [ ]:
Weekly_Sales_All = df1.groupby(['Year', 'Week Number']).agg({'Price Each': 'sum', 'Quantity Ordered': 'sum', 'Turnover': 'sum', 'Margin': 'sum', 'Total Sales': 'sum'}).reset_index()
Weekly_Sales_All.tail()

In [ ]:
Weekly_Sales_All.loc[52,'Week Number'] = 53
Weekly_Sales_All.tail()

In [ ]:
Monthly_Sales_All = df1.groupby(['Year', 'Month']).agg({'Price Each': 'sum', 'Quantity Ordered': 'sum', 'Turnover': 'sum', 'Margin': 'sum', 'Total Sales': 'sum'}).reset_index()
Monthly_Sales_All['Average Purchase Cost'] = Monthly_Sales_All['Total Sales'] / Monthly_Sales_All['Quantity Ordered']
Monthly_Sales_All

In [ ]:
df1

## Data Exporation

In [ ]:
for col in df1.columns:
    print(col)

In [ ]:
df1.info()

In [ ]:
df1.describe().round(2)

In [ ]:
df1.describe(include = "all").round(2)

In [ ]:
df1["Category"].unique()

In [ ]:
df1["Category"].replace('Électronique', 'Electronics', inplace=True)
df1["Category"].replace('Alimentation', 'Food', inplace=True)
df1["Category"].replace('Vêtements', 'Clothes', inplace=True)

In [ ]:
df1["Category"].unique()

In [ ]:
df1["Product"].unique()

In [ ]:
df1["City"].unique()

In [ ]:
df1["State Abrev"].unique()

In [ ]:
df1["ZIP"].unique()

In [ ]:
df1["Year"].unique()

### Data Viz Exploration

In [ ]:
df1.pivot_table(values="Total Sales", index=["Product"], columns="Category", aggfunc='sum', fill_value=None, margins=True, dropna=True, margins_name='All')

In [ ]:
fig1 = go.Figure(data=go.Scatter(x=pd.to_datetime(Monthly_Sales_All['Year'].astype(str) + '-' + Monthly_Sales_All['Month'].astype(str) + '-01'),
                                 y=Monthly_Sales_All['Price Each'],
                                 mode='lines+markers'))

fig1.update_layout(title='Aggregated Monthly Sales Over Time',
                   xaxis_title='Time',
                   yaxis_title='Total Sales',
                   xaxis=dict(showgrid=True, showline=True, showticklabels=True),
                   yaxis=dict(showgrid=True, showline=True, showticklabels=True))

fig1.show()
pio.write_html(fig1, file='figure.html', auto_open=True)

In [ ]:
fig2 = go.Figure(data=go.Scatter(x=Weekly_Sales_All['Week Number'],
                                 y=Weekly_Sales_All['Price Each'],
                                 mode='lines+markers'))

fig2.update_layout(title='Aggregated Weekly Sales Over Time',
                   xaxis_title='Weeks',
                   yaxis_title='Total Sales',
                   xaxis=dict(showgrid=True, showline=True, showticklabels=True),
                   yaxis=dict(showgrid=True, showline=True, showticklabels=True))

fig2.show()
pio.write_html(fig2, file='figure.html', auto_open=True)

In [ ]:
fig3 = go.Figure(data=go.Scatter(x=pd.to_datetime(Daily_Sales_All['Year'].astype(str) + '-' + Daily_Sales_All['Month'].astype(str) + '-' + Daily_Sales_All['Day'].astype(str) + '-01'),
                                 y=Daily_Sales_All['Price Each'],
                                 mode='lines+markers'))

fig3.update_layout(title='Aggregated Daily Sales Over Time',
                   xaxis_title='Time',
                   yaxis_title='Total Sales',
                   xaxis=dict(showgrid=True, showline=True, showticklabels=True),
                   yaxis=dict(showgrid=True, showline=True, showticklabels=True))

fig3.show()
pio.write_html(fig3, file='figure.html', auto_open=True)

### Peak and Lowest Sales

In [ ]:
peak_sales = Daily_Sales_All[Daily_Sales_All['Price Each'] == Daily_Sales_All['Price Each'].max()]
low_sales = Daily_Sales_All[Daily_Sales_All['Price Each'] == Daily_Sales_All['Price Each'].min()]

print(f"Peak Sales were on Month: {peak_sales['Month'].values[0]}, Day: {peak_sales['Day'].values[0]}, {peak_sales['Year'].values[0]}")
print(f"Lowest Sales were on Month: {low_sales['Month'].values[0]}, Day: {peak_sales['Day'].values[0]}, {low_sales['Year'].values[0]}")

In [ ]:
Daily_Sales_All['Sales Growth'] = Daily_Sales_All['Price Each'].pct_change() * 100

# Plotting the growth rate
plt.figure(figsize=(12, 6))
sns.lineplot(x=pd.to_datetime(Daily_Sales_All['Year'].astype(str) + '-' + Daily_Sales_All['Month'].astype(str) + '-' + Daily_Sales_All['Day'].astype(str) + '-01'),
             y='Sales Growth', data=Daily_Sales_All)
plt.title('Daily Sales Growth Rate')
plt.ylabel('Growth Rate (%)')
plt.xlabel('Date')
plt.grid(True)
plt.show()

In [ ]:
Weekly_Sales_All['Sales Growth'] = Weekly_Sales_All['Price Each'].pct_change() * 100

# Plotting the growth rate
plt.figure(figsize=(12, 6))
sns.lineplot(x='Week Number',
             y='Sales Growth', data=Weekly_Sales_All)
plt.title('Weekly Sales Growth Rate')
plt.ylabel('Growth Rate (%)')
plt.xlabel('Week Number')
plt.grid(True)
plt.show()

In [ ]:
Monthly_Sales_All['Sales Growth'] = Monthly_Sales_All['Price Each'].pct_change() * 100

# Plotting the growth rate
plt.figure(figsize=(12, 6))
sns.lineplot(x=pd.to_datetime(Monthly_Sales_All['Year'].astype(str) + '-' + Monthly_Sales_All['Month'].astype(str) + '-01'),
             y='Sales Growth', data=Monthly_Sales_All)
plt.title('Monthly Sales Growth Rate')
plt.ylabel('Growth Rate (%)')
plt.xlabel('Date')
plt.grid(True)
plt.show()

## Total Sales by Category

In [ ]:
fig = px.box(df1,x="Category", y="Total Sales")
fig.show()

In [ ]:
state_sales = df1.groupby('State Abrev')['Total Sales'].sum().sort_values(ascending=True)

# Plotting sales by city
plt.figure(figsize=(12, 7))
state_sales.plot(kind='barh', color='DarkGreen')
plt.title('State Sales Distribution')
plt.ylabel('State')
plt.xlabel('Total Sales ($M)')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

In [ ]:
state_sales = df1.groupby('ZIP')['Total Sales'].sum().sort_values(ascending=True)

# Plotting sales by city
plt.figure(figsize=(12, 7))
state_sales.plot(kind='barh', color='DarkBlue')
plt.title('ZIP Code Distribution')
plt.ylabel('ZIP')
plt.xlabel('Total Sales ($M)')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

In [ ]:
state_sales = df1.groupby('Product')['Total Sales'].sum().sort_values(ascending=True)
plt.figure(figsize=(12, 7))
state_sales.plot(kind='barh', color='DarkRed')
plt.title('Product Sales Distribution')
plt.ylabel('Product')
plt.xlabel('Total Sales ($100,000)')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

In [ ]:
df6 = df1.groupby(['Product']).agg({'Quantity Ordered': 'sum', 'Total Sales': 'sum'}).sort_values(by=['Total Sales'], ascending=False).reset_index()
df6

### Correlation Matrix

In [ ]:
correlation_matrix = df1.corr()
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(correlation_matrix, annot=True, cmap="mako", fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
ax = sns.barplot(data = df1, 
                     x = 'Quantity Ordered', 
                     y = 'Total Sales')

### Top 5 Zip Codes for Sales

In [ ]:
heatmap_data = Monthly_Sales_All.pivot("Month", "Year", "Total Sales")
plt.figure(figsize=(6, 8))
sns.heatmap(heatmap_data, cmap="rocket_r", annot=True, fmt=",.0f")
plt.title('Monthly Sales Heatmap')
plt.show()

In [ ]:
df4 = df1.groupby(['ZIP', 'State Abrev', 'City']).agg({'Quantity Ordered': 'sum', 'Total Sales': 'sum'}).sort_values(by=['Total Sales'], ascending=False).reset_index()
df4.head()

In [ ]:
df5 = df1.groupby(['ZIP', 'State Abrev', 'City', 'Local Address']).agg({'Quantity Ordered': 'sum', 'Total Sales': 'sum'}).sort_values(by=['Total Sales'], ascending=False).reset_index()
df5

### Top 20 Local Addresses by Total Sales

In [ ]:
df7 = df5.sort_values(["Total Sales"], ascending=False)
df7.head(20)

### Top 20 Local Addresses by Quanity Ordered

In [ ]:
df8 = df5.sort_values(["Quantity Ordered"], ascending=False)
df8.head(20)